# 1.0 Case Study: constructing a network of wikipedia pages

In [1]:
!pip install wikipedia



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 62.6/62.6 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 96.7/96.7 kB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 123.6/123.6 kB ? eta 0:00:00
     -------------------------------------- 157.0/157.0 kB 9.8 MB/s eta 0:00:00
  Running setup.py install for wikipedia: started
  Running setup.py install for wikipedia: finished with status 'done'


  DEPRECATION: wikipedia is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from operator import itemgetter
import networkx as nx
import wikipedia
import matplotlib.pyplot as plt

In [3]:
nx.__version__

'3.1'

In [7]:
#Começando processo de geração da rede
#Página escolhida: https://pt.wikipedia.org/wiki/Isaac_Newton

SEED = "Isaac Newton".title()
STOPS = ("International Standard Serial Number",
         "International Standard Book Number",
         "National Diet Library",
         "International Standard Name Identifier",
         "International Standard Book Number (Identifier)",
         "Pubmed Identifier",
         "Pubmed Central",
         "Digital Object Identifier",
         "Arxiv",
         "Proc Natl Acad Sci Usa",
         "Bibcode",
         "Library Of Congress Control Number",
         "Jstor",
         "Doi (Identifier)",
         "Isbn (Identifier)",
         "Pmid (Identifier)",
         "Arxiv (Identifier)",
         "Bibcode (Identifier)")

#store both the layer to which a node belongs and the node name together as a tuple 
#on the todo_lst list.

todo_lst = [(0, SEED)] # The SEED is in the layer 0
todo_set = set(SEED) # The SEED itself
done_set = set() # Nothing is done yet

#Gerando grafo g

g = nx.DiGraph()
layer, page = todo_lst[0]

#the main collection/construction loop of the project.

while layer < 2:
  # Remove the name page of the current page from the todo_lst,
  # and add it to the set of processed pages.
  # If the script encounters this page again, it will skip over it.
  del todo_lst[0]
  done_set.add(page)

  # Show progress
  print(layer, page)

  # Attempt to download the selected page.
  try:
    wiki = wikipedia.page(page)
  except:
    print("Could not load", page)
    layer, page = todo_lst[0]
    continue

  for link in wiki.links:
    link = link.title()
    if link not in STOPS and not link.startswith("List Of"):
      if link not in todo_set and link not in done_set:
        todo_lst.append((layer + 1, link))
        todo_set.add(link)
      g.add_edge(page, link)
  layer, page = todo_lst[0]

0 Isaac Newton
1 'S Gravesande
1 1705 English General Election
1 1 John 5:7
Could not load 1 John 5:7
1 2Df Galaxy Redshift Survey
1 Asin (Identifier)
1 A History Of The Theories Of Aether And Electricity
1 A Priori And A Posteriori
1 Aaron Guerdon
Could not load Aaron Guerdon
1 Aaron Klug
1 Abraham De Moivre
1 Abraham De La Pryme
1 Absolute Space And Time
1 Abstract And Concrete
1 Abstract Object Theory
1 Abu'L Hasan Ibn Arfa Ra'A
1 Abu Bakr Al-Razi
1 Action At A Distance (Physics)
1 Action Theory (Philosophy)
1 Adam Ferguson
1 Adam Smith
1 Adam Weishaupt
1 Adamantios Korais
1 Adriaan Koerbagh
1 Adrian Smith (Statistician)
1 Agathodaemon (Alchemist)
1 Age Of Enlightenment
1 Age Of The Universe
1 Al-Jildaki
Could not load Al-Jildaki
1 Al-Simawi
1 Al-Tughrai
1 Al-Zahrawi
1 Alan Guth
1 Alan Lloyd Hodgkin
1 Albert Einstein
1 Albertus Magnus
1 Alchemical Symbols (Unicode Block)
1 Alchemical Symbol
Could not load Alchemical Symbol
1 Alchemist
1 Alchemy
1 Alchemy And Chemistry In The Medieva

1 Eugène Canseliet
1 Euler
1 Euler'S Summation Formula
1 Evangelista Torricelli
1 Evolutionism
1 Existence
1 Existentialism
1 Experience
1 Explanatory Power
1 F. J. Duarte
1 Fact
1 Fahrenheit
1 Faith And Rationality
1 Fallibilism
1 Falsifiability
1 Farad
Could not load Farad
1 Faraday (Unit)
1 Fasciculus Chemicus
1 Fellow Of The Royal Society
1 Feminist Metaphysics
1 Feminist Method
1 Ferdinando Galiani
1 Finite Difference
1 Finite Differences
1 Firmin Abauzit
1 Florian Cajori
1 Flower Of Kent
1 Fluxion
1 Foot-Lambert
1 Force
Could not load Force
1 Foundationalism
1 Foundations Of Science
1 Francesco Mario Pagano
1 Francis Bacon
Could not load Francis Bacon
1 Francis Hutcheson (Philosopher)
1 Francisco Suárez
1 François Hemsterhuis
1 François Quesnay
1 Frater Albertus
1 Frederick Gowland Hopkins
1 Frederik Van Leenhof
1 Free Market
1 Free Will
1 French Academy Of Sciences
1 Friedmann Equations
1 Friedmann–Lemaître–Robertson–Walker Metric
1 Friedrich Nietzsche
1 Friedrich Schiller
1 Ful

1 Leandro Fernández De Moratín
1 Leibniz-Clarke Correspondence
1 Leibniz–Newton Calculus Controversy
1 Lens (Optics)
1 Leyden Papyrus X
1 Liber Hermetis De Alchemia
1 Liber Ignium
1 Liber De Compositione Alchemiae
1 Liber Lucis
1 Libertarianism (Metaphysics)
1 Liberty
Could not load Liberty
1 Liberté, Égalité, Fraternité
1 Librivox
1 Lincolnshire
1 Linguistic Modality
1 Listed Building
1 Literary Review
1 Logarithms
1 Lord Charles Spencer
1 Loránd Eötvös
1 Louis Harold Gray
1 Louis De Jaucourt
1 Lucasian Professor Of Mathematics
1 Ludwig Wittgenstein
1 Luigi Galvani
1 Luminiferous Aether
1 Lumières
1 Lunar Theory
1 Mr (Identifier)
1 Mache (Unit)
1 Macquorn Rankine
1 Magical Thinking
1 Magister Salernus
1 Magnum Opus (Alchemy)
1 Maimonides
1 Manchester University Press
1 Mappae Clavicula
1 Marc Aaronson
1 Marie Curie
Could not load Marie Curie
1 Marquis De Condorcet
1 Marquis De Sade
1 Martin Bowes
1 Martin Folkes
1 Martin Heidegger
1 Martin Rees, Baron Rees Of Ludlow
1 Martín Sarmiento

1 Royal Society
1 Rudolf Carnap
1 Rudolf Steiner
1 Russian Enlightenment
1 Rutbat Al-Hakim
1 Réaumur Scale
1 S2Cid (Identifier)
1 Si Derived Unit
1 Salomon Trismosin
1 Samuel Clarke
1 Samuel Johnson
1 Samuel Langley
1 Samuel Norton (Alchemist)
1 Samuel Pepys
1 Samuel Von Pufendorf
1 Samuil Micu-Klein
1 Sapere Aude
1 Saul Kripke
1 Schrödinger–Newton Equation
1 Science
Could not load Science
1 Science Studies
1 Scientific Revolution
Could not load Scientific Revolution
1 Scientific Essentialism
1 Scientific Formalism
1 Scientific Law
1 Scientific Method
1 Scientific Pluralism
1 Scientific Realism
1 Scientific Skepticism
1 Scientific Theory
1 Scientism
1 Scott Berkun
1 Scottish Enlightenment
1 Sebastião José De Carvalho E Melo, 1St Marquis Of Pombal
1 Secretum Secretorum
1 Semantic View Of Theories
1 Series (Mathematics)
1 Seventy Books
1 Shape Of The Universe
1 Siemens (Unit)
1 Sievert
1 Silver Standard
1 Simon Stevin
Could not load Simon Stevin
1 Sinecure
1 Sir
Could not load Sir
1 Sir 

In [8]:
print("{} nodes, {} edges".format(len(g), nx.number_of_edges(g)))

125967 nodes, 429942 edges


In [10]:
#Processo de limpeza

# make a copy of raw graph
original = g.copy()






In [11]:
# remove self loops

#Aqui removi o loop que verificava páginas com o nome no plural, visto que por ser nome de pessoa,
# não há plural

g.remove_edges_from(nx.selfloop_edges(g))

duplicates = [(x, y) for x, y in
              [(node, node.replace("-", " ")) for node in g]
                if x != y and y in g]
print(duplicates)

for dup in duplicates:
  g = nx.contracted_nodes(g, *dup, self_loops=False)

# nx.contracted creates a new node/edge attribute called contraction
# the value of the attribute is a dictionary, but GraphML
# does not support dictionary attributes
nx.set_node_attributes(g, 0,"contraction")
nx.set_edge_attributes(g, 0,"contraction")

[('Anthony Ashley-Cooper, 3Rd Earl Of Shaftesbury', 'Anthony Ashley Cooper, 3Rd Earl Of Shaftesbury'), ('Charles-Augustin De Coulomb', 'Charles Augustin De Coulomb'), ('Jean-Jacques Rousseau', 'Jean Jacques Rousseau'), ('Jean-Paul Sartre', 'Jean Paul Sartre'), ('Lambda-Cdm Model', 'Lambda Cdm Model'), ('Pierre-Simon Laplace', 'Pierre Simon Laplace'), ('Theory-Ladenness', 'Theory Ladenness'), ('A-Theory Of Time', 'A Theory Of Time'), ('Thing-In-Itself', 'Thing In Itself'), ('Truth-Value', 'Truth Value'), ('Al-Farabi', 'Al Farabi'), ('Al-Kindi', 'Al Kindi'), ('Many-Worlds Interpretation', 'Many Worlds Interpretation'), ('Neo-Confucianism', 'Neo Confucianism'), ('Belles-Lettres', 'Belles Lettres'), ('Laissez-Faire', 'Laissez Faire'), ('Rent-Seeking', 'Rent Seeking'), ('Anthony Ashley-Cooper, 7Th Earl Of Shaftesbury', 'Anthony Ashley Cooper, 7Th Earl Of Shaftesbury'), ('Anne-Robert-Jacques Turgot, Baron De Laune', 'Anne Robert Jacques Turgot, Baron De Laune'), ('Claude-Adrien Helvétius', '

In [12]:
print("{} nodes, {} edges".format(len(g), nx.number_of_edges(g)))

125362 nodes, 429219 edges


In [13]:
#Truncate the network

# filter nodes with degree greater than or equal to 2
core = [node for node, deg in dict(g.degree()).items() if deg >= 2]

# select a subgraph with 'core' nodes
gsub = nx.subgraph(g, core)

print("{} nodes, {} edges".format(len(gsub), nx.number_of_edges(gsub)))

nx.write_graphml(gsub, "cna.graphml")


47521 nodes, 351378 edges
